In [ ]:
import sys
sys.path.append("..")

In [ ]:
from src import models

device = "cuda:5"
mt = models.load_model("gptj", device=device)

In [ ]:
from src import data

dataset = data.load_dataset()

In [ ]:
from src import operators

estimator = operators.JacobianIclMeanEstimator(mt=mt, h_layer=11, z_layer=26)
relation = dataset[0].set(
    samples=dataset[0].samples[:3],
    prompt_templates=[dataset[0].prompt_templates[0]],
)
operator = estimator(relation)

In [ ]:
operator("Italy").predictions

In [ ]:
from src import functional

import baukit
import torch

def apply_lm_head(hiddens):
    predictions = mt.lm_head[1:](hiddens).topk(dim=-1, k=10).indices
    return [
        [mt.tokenizer.decode(t) for t in ts]
        for ts in predictions
    ]


outputs = functional.compute_hidden_states(mt=mt, layers=[26], prompt=[
    "The capital of Norway is",
    "The capital of France is",
])
z = outputs.hiddens[0][0, -1]
z_old = outputs.hiddens[0][-1, -1]
print(z)
print(z_old)

# j_inv = torch.linalg.pinv(operator.weight.float()).half()
j_inv = torch.eye(4096).half().to(device)

delta = j_inv.mm(z[..., None] - operator.bias.t())
print(delta.norm())

delta_old = j_inv.mm(z_old[..., None] - operator.bias.t())
print(delta_old.norm())

def edit_output(output, layer):
    if output[0].shape[1] == 1:
        return output
    print(output[0][0, 3].norm())
    output[0][0, 3] = (
#         -1 * delta_old.squeeze() +
        delta.squeeze()
    )
    print(output[0][0, 3].norm())
    return output


inputs = mt.tokenizer("The capital of France is the city of", return_tensors="pt").to(device)
with baukit.TraceDict(mt.model, [f"transformer.h.{l}" for l in [11]], edit_output=edit_output):
    outputs = mt.model.generate(
        **inputs,
        pad_token_id=mt.tokenizer.eos_token_id,
        max_new_tokens=100,
    )
mt.tokenizer.batch_decode(outputs)[0]

In [ ]:
import torch

def apply_lm_head(hiddens):
    predictions = mt.lm_head[1:](hiddens).argmax(dim=-1)
    print(predictions.shape)
    return [(i, mt.tokenizer.decode(tok)) for i, tok in enumerate(predictions.tolist())]


@torch.inference_mode()
def logit_lens(prompt):
    inputs = mt.tokenizer(prompt, return_tensors="pt", padding="longest", truncation=True).to(device)
    inputs.pop("token_type_ids", None)
    outputs = mt.model(**inputs, return_dict=True, output_hidden_states=True)
    hiddens = torch.cat(outputs.hidden_states[1:], dim=0)[:, -1]
    return apply_lm_head(hiddens)


logit_lens("The capital of France is")